In [ ]:
#default_exp vision.core

Implement ideas from [kaggle competition](https://www.kaggle.com/c/rsna-intracranial-hemorrhage-detection/discussion/117242) which can be used for any sequenced imaging modeling.

In [ ]:
#export
from fastai.vision.all import *
from fastai.medical.imaging import *

Assuming we already have representations for each slice in a sequence, let's start off from AWD-LSTM

In [ ]:
bs, sl, dim = 32, 16, 1024
x = torch.randn(bs,sl,dim); x.shape

### SequenceClassifierA

In [ ]:
#export
class SequenceClassifierA(Module):
    "Predicts for each sequence, e.g. no pooling"
    def __init__(self, emb_sz, n_hid, n_out, bidir=True, ps=0.3, rnn=nn.LSTM):
        self.rnn1 = rnn(emb_sz, n_hid, bidirectional=True)
        self.rnn2 = rnn(n_hid*2, n_hid//2, bidirectional=True)
        dims = [n_hid, n_hid//2, n_out]
        ps = [ps]*len(dims)
        acts = [nn.ReLU(inplace=True)] * (len(dims) - 2) + [None]
        layers = [LinBnDrop(i, o, p=p, act=a, bn=False) for i,o,p,a in zip(dims[:-1], dims[1:], ps, acts)]
        self.cls_head = nn.Sequential(*layers)
    
    def forward(self, x):
        x, _ = self.rnn1(x)
        x, _ = self.rnn2(x)
        return self.cls_head(x)

In [ ]:
seq_model = SequenceClassifierA(dim, n_hid=256, n_out=1, rnn=nn.GRU)
out = seq_model(x).sigmoid()
assert out.shape == torch.Size([32,16,1])

In [ ]:
seq_model

### SequenceClassifierB

In [ ]:
# https://github.com/darraghdog/rsna/blob/a97018a7b7ec920425189c7e37c1128dd9cb0158/scripts/resnext101v12/trainlstmdeltasum.py#L352

In [ ]:
class SpatialDropout(nn.Dropout2d):
    def forward(self, x):
        x = x.unsqueeze(2)    # (N, T, 1, K)
        x = x.permute(0, 3, 2, 1)  # (N, K, 1, T)
        x = super(SpatialDropout, self).forward(x)  # (N, K, 1, T), some features are masked
        x = x.permute(0, 3, 2, 1)  # (N, T, 1, K)
        x = x.squeeze(2)  # (N, T, K)
        return x

In [ ]:
class SequenceClassifierB(Module):
    def __init__(self, dim, n_hid=64, p = 0.3, n_class=2):
        
        self.embedding_dropout = SpatialDropout(p)
        
        self.lstm1 = nn.LSTM(dim, n_hid, bidirectional=True, batch_first=True)
        self.lstm2 = nn.LSTM(n_hid * 2, n_hid, bidirectional=True, batch_first=True)

        self.linear1 = nn.Linear(n_hid*2, n_hid*2)
        self.linear2 = nn.Linear(n_hid*2, n_hid*2)

        self.linear = nn.Linear(n_hid*2, n_class)

    def forward(self, x, lengths=None):
        
        x_add = torch.cat((x, x), -1)
        
        h_lstm1, _ = self.lstm1(x)
        h_lstm2, _ = self.lstm2(h_lstm1)
        
        h_conc_linear1  = F.relu(self.linear1(h_lstm1))
        h_conc_linear2  = F.relu(self.linear2(h_lstm2))
        
        hidden = h_lstm1 + h_lstm2 + h_conc_linear1 + h_conc_linear2 + x_add

        output = self.linear(hidden)
        
        return output

In [ ]:
seq_model = SequenceClassifierB(dim, n_hid=1024, n_class=1)
out = seq_model(x).sigmoid()
assert out.shape == torch.Size([32,16,1])

In [ ]:
class SequenceClassifierB(Module):
    def __init__(self, model_num, feature_dim, drop_out, feature_num=128, hidden=96, lstm_layers=2, add_position=True):
        # seq model 1
        self.fea_conv = nn.Sequential(nn.Dropout2d(drop_out),
                                      nn.Conv2d(feature_dim, 512, kernel_size=(1, 1), stride=(1,1),padding=(0,0), bias=False),
                                      nn.BatchNorm2d(512),
                                      nn.ReLU(),
                                      nn.Dropout2d(drop_out),
                                      nn.Conv2d(512, 128, kernel_size=(1, 1), stride=(1, 1), padding=(0, 0), bias=False),
                                      nn.BatchNorm2d(128),
                                      nn.ReLU(),
                                      nn.Dropout2d(drop_out),
                                      )

        self.fea_first_final = nn.Sequential(nn.Conv2d(128*feature_num, 6, kernel_size=(1, 1), stride=(1, 1), padding=(0, 0), bias=True))
    
    def forward(self, x):
        
        fea = self.fea_conv(fea)
        fea = fea.permute(0, 1, 3, 2).contiguous()
        fea = fea.view(batch_size, 128 * feature_num, -1).contiguous()
        fea = fea.view(batch_size, 128 * feature_num, -1, 1).contiguous()
        fea_first_final = self.fea_first_final(fea)
        #################################################
        out0 = fea_first_final.permute(0, 3, 2, 1)

In [ ]:
class SequenceClassifierB(Module):
    def __init__(self, model_num, feature_dim, drop_out, feature_num=128, hidden=96, lstm_layers=2, add_position=True):
        # seq model 1
        self.fea_conv = nn.Sequential(nn.Dropout2d(drop_out),
                                      nn.Conv2d(feature_dim, 512, kernel_size=(1, 1), stride=(1,1),padding=(0,0), bias=False),
                                      nn.BatchNorm2d(512),
                                      nn.ReLU(),
                                      nn.Dropout2d(drop_out),
                                      nn.Conv2d(512, 128, kernel_size=(1, 1), stride=(1, 1), padding=(0, 0), bias=False),
                                      nn.BatchNorm2d(128),
                                      nn.ReLU(),
                                      nn.Dropout2d(drop_out),
                                      )

        self.fea_first_final = nn.Sequential(nn.Conv2d(128*feature_num, 6, kernel_size=(1, 1), stride=(1, 1), padding=(0, 0), bias=True))

        # # bidirectional GRU
        self.hidden_fea = hidden
        self.fea_lstm = nn.GRU(128*feature_num, self.hidden_fea, num_layers=lstm_layers, batch_first=True, bidirectional=True)
        self.fea_lstm_final = nn.Sequential(nn.Conv2d(1, 6, kernel_size=(1, self.hidden_fea*2), stride=(1, 1), padding=(0, 0), dilation=1, bias=True))

        ratio = 4
        if add_position:
            model_num += 2
        else:
            model_num += 1

        # seq model 2
        self.conv_first = nn.Sequential(nn.Conv2d(model_num, 128*ratio, kernel_size=(5, 1), stride=(1,1),padding=(2,0),dilation=1, bias=False),
                                        nn.BatchNorm2d(128*ratio),
                                        nn.ReLU(),
                                        nn.Conv2d(128*ratio, 64*ratio, kernel_size=(3, 1), stride=(1, 1), padding=(2, 0),dilation=2, bias=False),
                                        nn.BatchNorm2d(64*ratio),
                                        nn.ReLU())

        self.conv_res = nn.Sequential(nn.Conv2d(64 * ratio, 64 * ratio, kernel_size=(3, 1), stride=(1, 1),padding=(4, 0),dilation=4, bias=False),
                                      nn.BatchNorm2d(64 * ratio),
                                      nn.ReLU(),
                                      nn.Conv2d(64 * ratio, 64 * ratio, kernel_size=(3, 1), stride=(1, 1),padding=(2, 0),dilation=2, bias=False),
                                      nn.BatchNorm2d(64 * ratio),
                                      nn.ReLU(),)

        self.conv_final = nn.Sequential(nn.Conv2d(64*ratio, 1, kernel_size=(3, 1), stride=(1, 1), padding=(1, 0), dilation=1,bias=False))

        # bidirectional GRU
        self.hidden = hidden
        self.lstm = nn.GRU(64*ratio*6, self.hidden, num_layers=lstm_layers, batch_first=True, bidirectional=True)
        self.final = nn.Sequential(nn.Conv2d(1, 6, kernel_size=(1, self.hidden*2), stride=(1, 1), padding=(0, 0), dilation=1, bias=True))


    def forward(self, fea, x):
        batch_size, _, _, _ = x.shape

        fea = self.fea_conv(fea)
        fea = fea.permute(0, 1, 3, 2).contiguous()
        fea = fea.view(batch_size, 128 * feature_num, -1).contiguous()
        fea = fea.view(batch_size, 128 * feature_num, -1, 1).contiguous()
        fea_first_final = self.fea_first_final(fea)
        #################################################
        out0 = fea_first_final.permute(0, 3, 2, 1)
        #################################################

        # bidirectional GRU
        fea = fea.view(batch_size, 128 * feature_num, -1).contiguous()
        fea = fea.permute(0, 2, 1).contiguous()
        fea, _ = self.fea_lstm(fea)
        fea = fea.view(batch_size, 1, -1, self.hidden_fea * 2)
        fea_lstm_final = self.fea_lstm_final(fea)
        fea_lstm_final = fea_lstm_final.permute(0, 3, 2, 1)
        #################################################
        out0 += fea_lstm_final
        #################################################

        out0_sigmoid = torch.sigmoid(out0)
        x = torch.cat([x, out0_sigmoid], dim = 1)
        x = self.conv_first(x)
        x = self.conv_res(x)
        x_cnn = self.conv_final(x)
        #################################################
        out = x_cnn
        #################################################

        # bidirectional GRU
        x = x.view(batch_size, 256, -1, 6)
        x = x.permute(0,2,1,3).contiguous()
        x = x.view(batch_size, x.size()[1], -1).contiguous()
        x, _= self.lstm(x)
        x = x.view(batch_size, 1, -1, self.hidden*2)
        x = self.final(x)
        x = x.permute(0,3,2,1)
        #################################################
        out += x
        #################################################
        #res
        return out, out0

In [ ]:
seq_model = SequenceClassifierB(model_num=5, feature_dim=dim, drop_out=0.5)
out = seq_model(x).sigmoid()
assert out.shape == torch.Size([32,16,1])

In [ ]:
# https://www.kaggle.com/bminixhofer/speed-up-your-rnn-with-sequence-bucketing
class SequenceModel(Module):
    def __init__(self, emb_sz, n_hid=64, n_classes):
        
        
        self.lstm1 = nn.LSTM(emb_sz, n_hid, bidirectional=True, batch_first=True)
        self.lstm2 = nn.LSTM(n_hid * 2, n_hid, bidirectional=True, batch_first=True)

        self.linear1 = nn.Linear(n_hid*2, n_hid*2)
        self.linear2 = nn.Linear(n_hid*2, n_hid*2)

        self.linear = nn.Linear(n_hid*2, n_classes)

    def forward(self, x, lengths=None):

        h_embadd = torch.cat((h_embedding[:,:,:2048], h_embedding[:,:,:2048]), -1)
        
        h_lstm1, _ = self.lstm1(h_embedding)
        h_lstm2, _ = self.lstm2(h_lstm1)
        
        h_conc_linear1  = F.relu(self.linear1(h_lstm1))
        h_conc_linear2  = F.relu(self.linear2(h_lstm2))
        
        hidden = h_lstm1 + h_lstm2 + h_conc_linear1 + h_conc_linear2 + h_embadd

        output = self.linear(hidden)
        
        return output

In [ ]:
conv = nn.Conv1d(1024, 128, 3, 1, 1)

In [ ]:
conv(x.permute(0,2,1)).shape

In [ ]:
# https://www.kaggle.com/bminixhofer/speed-up-your-rnn-with-sequence-bucketing
class NeuralNet(nn.Module):
    def __init__(self, embed_size=trnemb.shape[-1]*3, LSTM_UNITS=64, DO = 0.3):
        super(NeuralNet, self).__init__()
        
        self.embedding_dropout = SpatialDropout(0.0) #DO)
        
        self.lstm1 = nn.LSTM(embed_size, LSTM_UNITS, bidirectional=True, batch_first=True)
        self.lstm2 = nn.LSTM(LSTM_UNITS * 2, LSTM_UNITS, bidirectional=True, batch_first=True)

        self.linear1 = nn.Linear(LSTM_UNITS*2, LSTM_UNITS*2)
        self.linear2 = nn.Linear(LSTM_UNITS*2, LSTM_UNITS*2)

        self.linear = nn.Linear(LSTM_UNITS*2, n_classes)

    def forward(self, x, lengths=None):
        h_embedding = x

        h_embadd = torch.cat((h_embedding[:,:,:2048], h_embedding[:,:,:2048]), -1)
        
        h_lstm1, _ = self.lstm1(h_embedding)
        h_lstm2, _ = self.lstm2(h_lstm1)
        
        h_conc_linear1  = F.relu(self.linear1(h_lstm1))
        h_conc_linear2  = F.relu(self.linear2(h_lstm2))
        
        hidden = h_lstm1 + h_lstm2 + h_conc_linear1 + h_conc_linear2 + h_embadd

        output = self.linear(hidden)
        
        return output

In [ ]:
m1 = AWD_LSTM_2(emb_sz=128, n_hid=1024, n_out=1024, n_layers=1, bidir=True)
l1 = nn.Linear(1024, 1024)
m2 = AWD_LSTM_2(emb_sz=1024, n_hid=1024, n_out=1024, n_layers=1, bidir=True)
l2 = nn.Linear(1024, 1024)

In [ ]:
m2

In [ ]:
output = m(x)

In [ ]:
output.shape